In [3]:
import langchain

In [ ]:
import os
api_key = ''
os.environ["HUGGINGFACEHUB_ACCESS_TOKEN"] = ""

In [5]:
!pip install -q youtube-transcript-api langchain-community langchain-huggingface faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127

In [6]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [ ]:
#indexing

In [9]:
video_id = '4SNThp0YiU4'
try :
  transcript_list = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
  transcript = " ".join(chunk['text'] for chunk in transcript_list)
  print(transcript)
except TranscriptsDisabled :
  print("no caption for this video")

- I've gathered 50 of the
biggest YouTubers on the planet. And whichever one lasts
the longest in this cube wins that million dollars
for their subscribers. Never before seen in YouTube history, 1 billion subscribers worth of YouTubers are competing for $1 million. And the rules are simple. All right, point the camera
down, close the line. - Oh!
- Okay. So if you step on the red line, you lose your subscribers
a million dollars. So be careful. Last to leave wins it all. - Like, bro, I
watched the "Squid Games" like reality show, it's like... - I feel like I'm just a target. - People are not
going to be friends in this box. - There's no way.
- This is our money, bro. - This is real USD. - I'm pumped, bro. - You finally made in a video.
- Finally. - The best strategy is
to just conserve energy, think about that million dollar pot. - Between us, would you
eliminate someone else so you could stay in? - For my subscribers, of course. - Yeah, yeah.
- No, no, no. Get out, no, no, no, no. Get 

In [10]:
len(transcript)

38818

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=120)
chunks = splitter.create_documents([transcript])

In [12]:
chunks[55]

Document(metadata={}, page_content="eliminated cases go. - This is so heartbreaking. - Just don't hit a\nkangaroo on your way home. - I don't like you, Nolan. - And the brutality of these cases didn't stop there. No steal. Because the most brutal was yet to come. Oh my gosh. - I feel stabbed, I do. - Yeah, that was crazy. - I don't want to lie. - You don't want to look at her\nin the eyes when she opens it? - Oh god, it says eliminated. - I can't do this, it's too hard.")

In [13]:
len(chunks)

102

In [14]:
embedding = HuggingFaceEndpointEmbeddings(
    repo_id = 'intfloat/e5-small-v2'
)

In [ ]:
embedding = HuggingFaceEndpointEmbeddings(
    repo_id='sentence-transformers/all-MiniLM-L6-v2',
    huggingfacehub_api_token=api_key
)

In [16]:
vector_store = FAISS.from_documents(chunks,embedding)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [17]:
vector_store.index_to_docstore_id

{0: '2af7e3eb-b33f-4af3-ba19-53b4f584fc6e',
 1: '79961026-9aed-4199-bfd3-becde9024b02',
 2: 'fbf5a0ba-721a-4a2e-ba7a-47c424afa985',
 3: 'a0e8ec03-905c-4be7-afcc-7948e77960c6',
 4: 'db5920bf-bed6-4194-924d-264aba528e09',
 5: '0a64cc05-030e-4ce7-b95f-a3ef7f6ab10a',
 6: '7c7ba633-14a2-43b6-9312-e59000841fb4',
 7: '92e5bda8-8335-47e7-9757-5199f43f6b8f',
 8: '9f01862c-3637-47bd-8a8a-9eed843a88d5',
 9: '8de451e9-f31d-4b6c-b621-004e71863812',
 10: 'a74bf634-b475-4774-9e9d-3218c5491a4a',
 11: '7f3d6efc-5b03-498b-829f-f063419a9952',
 12: '4efb397e-3406-4ef1-9b12-1620f4cba43d',
 13: '6c009f3d-3604-44d5-ae59-5f56ad234bf8',
 14: 'a6cc87da-c354-4f9c-bbd8-6bf1b0eb5b0c',
 15: '0a7b5796-edb0-48d3-8f79-781a234e7f8a',
 16: 'f39c65b4-b1e2-4e9b-90b3-36e7b1e587d6',
 17: '3a03c15d-9c41-495f-bd1b-98c2f7073534',
 18: '5d8107e2-07df-4c0c-9605-67e658f8e5e6',
 19: '0430855a-35be-42f5-a86a-45d1bbb3813a',
 20: 'bc9bac31-af24-42a8-a664-986b84604dab',
 21: 'd5b6f671-ed3e-4169-a34e-ae594c4a7b9c',
 22: '096d7aa5-c711-

In [18]:
vector_store.get_by_ids(['b57edc89-d92b-4970-8449-11530f231cd5'])

[]

In [19]:
#retrival

In [36]:
retriever = vector_store.as_retriever(search_kwargs={'k':2},search_type='similarity')

In [37]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEndpointEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e12a8bdbf50>, search_kwargs={'k': 2})

In [22]:
retriever.invoke("who is the winner of the video")

[Document(id='c34ae29b-5316-4ece-ba62-3f8ed1526fe2', metadata={}, page_content="24 hours of this challenge. And I'm going to be honest,\nit's actually pretty crazy. All these YouTubers are\njust casually sleeping next to $1 million in cash. And by the end of this video, it could quite literally be yours. - Hey, yo. Good morning, Logang. What's popping? - Oh my God, that takes me back. I'm getting tingles, bro. - Ohayo. - Ohio? - It's good morning. - Ohayo. - Time to lock in and do\nit for the subscribers. - Good morning."),
 Document(id='67a72aef-a76d-4f0d-b5fe-9e55289e1aa0', metadata={}, page_content="thinned and I'm going to win. - And while setting\nup the next challenge, I let the 34 remaining\nYouTubers entertain themselves. - We can play a game. How close can you get to that line, bro? - We can do that with bottle caps. - That's right,\nthe device that entertains your favorite creators are bottle caps. - Oh, whoa.\n- Whoa. - Hey, watch this, watch this. - Not going to think about

In [23]:
#augumentation

In [ ]:
llmn = HuggingFaceEndpoint(
    repo_id = 'HuggingFaceH4/zephyr-7b-beta',
    task = 'text-generation',
    huggingfacehub_api_token = api_key,
)


model = ChatHuggingFace(
    llm = llmn,
    verbose = True
)

In [25]:
prompt = PromptTemplate(
    template = '''
    you are a very helpful assistant.
    answer only from the provided transcript context.
    if the context is insufficient, just say you don't know.

    {context}
    question : {question}
    ''',
    input_variables = ['context','question']
)

In [26]:
question = 'which youtuber won 1 million doller, winner of this video'
retriveved_docs = retriever.invoke(question)

In [27]:
context_text = '\n\n'.join(doc.page_content for doc in retriveved_docs)

In [28]:
final_prompt = prompt.invoke({'context':context_text,'question':question})

In [29]:
final_prompt

StringPromptValue(text="\n    you are a very helpful assistant.\n    answer only from the provided transcript context.\n    if the context is insufficient, just say you don't know.\n\n    - I've gathered 50 of the\nbiggest YouTubers on the planet. And whichever one lasts\nthe longest in this cube wins that million dollars\nfor their subscribers. Never before seen in YouTube history, 1 billion subscribers worth of YouTubers are competing for $1 million. And the rules are simple. All right, point the camera\ndown, close the line. - Oh!\n- Okay. So if you step on the red line, you lose your subscribers\na million dollars. So be careful. Last to leave wins it all. - Like, bro, I\n\nfor the player with the money, the game is over and\nthat YouTuber walks away with the million dollars\nfor their subscribers. This is a game of bluffing and how well you can read each other. Which one of you has the best poker face? Start the timer. One of you can win a million dollars in the next few minutes. 

In [ ]:
# generation

In [ ]:
result = model.invoke(final_prompt)
print(result.content)

Unfortunately, the given transcript context does not reveal who won the million dollars in this YouTube challenge. The video ends with the statement "the last tower standing was without a doubt, the most competitive game we had," implying that the winner may have emerged from that particular challenge, but it is not explicitly stated.


#Building Chain

In [38]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [39]:
def format_docs(retriveved_docs):
  context_text = '\n\n'.join(doc.page_content for doc in retriveved_docs)
  return context_text

In [40]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question':RunnablePassthrough()
})

In [43]:
parallel_chain.invoke('youtuber carrymination aka ajay nager mension in this video')

{'context': "it's for the YouTube community. Whoever is the first\nneeds to be respected. - Let's go.\n- I've been watching since 2014. - All right, you know what? - I am very happy - Oh, now I wish you would've won. The last tower standing\nwas without a doubt, the most competitive game we had. Pull that bad boy. These two were at it for\nmore than two hours longer than the rest of the YouTubers. - That's crazy. Oh. No, no, no. - You got it, yo. - No.\n\n- I've gathered 50 of the\nbiggest YouTubers on the planet. And whichever one lasts\nthe longest in this cube wins that million dollars\nfor their subscribers. Never before seen in YouTube history, 1 billion subscribers worth of YouTubers are competing for $1 million. And the rules are simple. All right, point the camera\ndown, close the line. - Oh!\n- Okay. So if you step on the red line, you lose your subscribers\na million dollars. So be careful. Last to leave wins it all. - Like, bro, I",
 'question': 'youtuber carrymination aka a

In [44]:
parser = StrOutputParser()

In [45]:
main_chain = parallel_chain | prompt | model | parser

In [47]:
main_chain.invoke('can you samarize the video')

'The video promises to be the most brutal and intense challenge for popular YouTubers. The first challenge involves drawing a curtain, which leads to chaos and boredom among the participants. Some of the YouTubers leave the area, but four have already departed, and one more needs to go before the next challenge can begin. The viewer is left wondering what the next challenge will involve. Overall, the video sets up an anticipation for something extreme and unexpected.'

In [64]:
def fetch_video_id(url):
  video_id = url.split('=')[1]
  return video_id

def format_docs(retriveved_docs):
  context_text = '\n\n'.join(doc.page_content for doc in retriveved_docs)
  return context_text

In [ ]:
def youtube_chatbot(url,question):
  # fetch the video id of url
  video_id = fetch_video_id(url)
  # print(video_id)
  # fetch youtube video transcript
  video_id = 'g6tlNyr5sl8'
  try :
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id,languages=['en'])
    transcript = " ".join(chunk['text'] for chunk in transcript_list)
    # print(transcript)
  except TranscriptsDisabled :
    print("no caption for this video")

  # text splitter
  splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 200)
  chunks = splitter.create_documents([transcript])

  # embedding model
  embedding = HuggingFaceEndpointEmbeddings(
      repo_id = 'sentence-transformers/all-MiniLM-L6-v2',
      huggingfacehub_api_token = api_key
  )
  # llm
  llm = HuggingFaceEndpoint(
      repo_id = 'HuggingFaceH4/zephyr-7b-beta',
      task = 'text-generation',
      huggingfacehub_api_token = api_key
  )
  # chatmodel
  model = ChatHuggingFace(
      llm = llm,
      verbose = True
  )
  # prompt template
  prompt = PromptTemplate(
      template = '''
      you are a very helpful assistant.
      answer only from the provided transcript context.
      if the context is insufficient, just say you don't know.

      {context}
      question : {question}
      ''',
      input_variables=['context','question']
  )
  # vector store
  vector_store = FAISS.from_documents(chunks,embedding)
  # retriver
  retriever = vector_store.as_retriever(search_kwargs={'k':3}, search_type = 'similarity')
  # create parallel chain through runnable
  parallel_chain = RunnableParallel({
      'context': retriever | RunnableLambda(format_docs),
      'question': RunnablePassthrough()
  })
  # string output parser
  parser = StrOutputParser()

  main_chain = parallel_chain | prompt | model | parser
  result = main_chain.invoke(question)
  print(result)
  return result

In [80]:
youtube_chatbot(url='https://www.youtube.com/watch?v=g6tlNyr5sl8',question="generate the summary of the video")

This is me on top of a speeding train. And this is me in a hot air balloon to answer some
of the most debated science questions on the internet. For example, my hot air balloon pilot here says
he can land me anywhere I want, and yet there's nothing here that resembles a steering wheel,
which begs the question, how do you actually steer these things? - So, we steer... - And now I'm on top of the train to learn once
and for all, why do you land in the same spot if you jump inside the train but not
if you jump on top of the train? But I'm not stopping there. (screaming) - Because today we're gonna investigate 5 more
physics and engineering puzzles using simple demonstrations as we go. - Because our goal by the end of this video is not
for you just to know the right answers, but more importantly for you to
understand why they're the right answers. To kick things off, I've got a science
riddle for you that most people get wrong. What is the first man-made
object to break the sound barrier? 

"The video explores the idea of using a treadmill to launch a plane from a short runway in one's backyard. The scientific basis behind this is explained, highlighting that planes need to reach a certain speed to take off, which requires a specific length of runway. The video introduces the concept of wind direction and speed changing at different heights through balloons being launched in 1000 locations worldwide at specific times. The experiment is conducted using a small-scale model, but the aircraft fails to take off despite ramping up the treadmill's speed. The video questions if this backyard treadmill idea is feasible, highlighting the need for airflow around the wings to create lift. Overall, the summary is that the video presents an imaginative concept to launch a plane from a short runway using a treadmill, but the scientific principles behind plane takeoffs and wind direction prevent it from being a viable solution."